In [ ]:
import cv2
from pynq.lib.video import *
from pynq.overlays.base import BaseOverlay

In [ ]:
## Format frame for HDMI output ##

def format_frame(hdmi_out, frame):
    
    # Read input frame size
    img_h = frame.shape[0]
    img_w = frame.shape[1]
    
    # Create new video frame
    outframe = hdmi_out.newframe()
    
    # Fill new video frame with zero
    zero_img = np.zeros((480, 640), dtype=np.uint8)
    outframe[0:480,0:640] = zero_img[0:480,0:640]

    # Transfer input frame to the video frame
    # For Gray imge
    outframe[0:img_h,0:img_w] = frame[0:img_h,0:img_w]
    
    return outframe

In [ ]:
# Load the overlay
base = BaseOverlay("base.bit")

In [ ]:
# Initialize HDMI I/O
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

# Configure HDMI input to gray scale
hdmi_in.configure(PIXEL_GRAY)
hdmi_in.start()

# Configure Output mode (w, h, bit per pixel)
hdmi_out_mode = VideoMode(640,480,8)
hdmi_out.configure(hdmi_out_mode, PIXEL_GRAY)
hdmi_out.start()

In [ ]:
while(True):
    # Read frame from HDMI input
    frame = hdmi_in.readframe()
    
    # Resize img size
    # Half of 1280 * 720
    # 1/3 of 1920 * 1080
    resized = cv2.resize(frame, (640,360)) 

    # Format frame for HDMI output
    outframe = format_frame(hdmi_out, resized)
    
    # Output frame to HDMI
    hdmi_out.writeframe(outframe)   

In [ ]:
# Clear all
hdmi_out.stop()
hdmi_in.stop()
del hdmi_in, hdmi_out